In [ ]:
class DQNetwork(nn.Module):
    """Deep Q-Network implemented in PyTorch."""
    def __init__(self, state_size, action_size):
        super(DQNetwork, self).__init__()
        self.fc1 = nn.Linear(state_size * state_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, action_size)

    def forward(self, x):
        x = torch.flatten(x, start_dim=1)  # Flatten the input
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [ ]:
class ReplayBuffer:
    def __init__(self, max_size):
        self.buffer = deque(maxlen=max_size)

    def push(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done))

    def sample(self, batch_size):
        states, actions, rewards, next_states, dones = zip(*random.sample(self.buffer, batch_size))
        return (
            torch.tensor(states, dtype=torch.float, device=device),
            torch.tensor(actions, dtype=torch.long, device=device),
            torch.tensor(rewards, dtype=torch.float, device=device),
            torch.tensor(next_states, dtype=torch.float, device=device),
            torch.tensor(dones, dtype=torch.float, device=device),
        )

    def __len__(self):
        return len(self.buffer)


# DQNetwork 클래스

## 개요
`DQNetwork` 클래스는 PyTorch를 기반으로 구현된 심층 Q-네트워크(Deep Q-Network, DQN)입니다. 이 네트워크는 강화학습에서 에이전트의 Q-값을 계산하는 데 사용됩니다.

## 메소드
### `__init__(self, state_size, action_size)`
- 입력 상태 크기와 가능한 행동 수를 기반으로 신경망을 초기화합니다.
- 네트워크는 세 개의 선형 계층으로 구성되어 있습니다.

### `forward(self, x)`
- 신경망을 통과하는 입력 x를 처리하여 각 행동에 대한 Q-값을 출력합니다.
- 입력 데이터는 평탄화되고, 각 계층은 ReLU 활성화 함수를 사용하여 처리됩니다.

# ReplayBuffer

## 주요 기능
- **경험 저장**:
  - 환경과의 상호작용에서 발생하는 `(state, action, reward, next_state, done)` 데이터를 저장.
  - 저장 공간의 크기를 제한하여 오래된 데이터를 삭제(FIFO 방식).
- **배치 샘플링**:
  - 저장된 데이터에서 무작위로 `batch_size`만큼 샘플링하여 학습 데이터를 제공합니다.
- **현재 크기 확인**:
  - 저장된 데이터의 개수를 반환.

---

## 클래스 메서드

### `__init__(self, max_size)`
- **설명**:
  - 버퍼를 초기화하고 최대 크기를 설정합니다.
- **파라미터**:
  - `max_size` (int): 버퍼의 최대 크기. 초과 시 오래된 데이터를 삭제.
- **구성 요소**:
  - `self.buffer`: `deque`를 사용하여 FIFO 방식으로 데이터를 저장.

---

### `push(self, state, action, reward, next_state, done)`
- **설명**:
  - 새로운 경험 데이터를 버퍼에 저장합니다.
- **파라미터**:
  - `state`: 현재 상태.
  - `action`: 에이전트가 선택한 행동.
  - `reward`: 행동 수행 후 받은 보상.
  - `next_state`: 행동 수행 후 도달한 다음 상태.
  - `done`: 에피소드 종료 여부 (`True` 또는 `False`).
- **특징**:
  - 최대 크기를 초과하면 가장 오래된 데이터가 삭제됩니다.

---

### `sample(self, batch_size)`
- **설명**:
  - 저장된 경험 데이터에서 `batch_size`만큼 랜덤 샘플링합니다.
- **파라미터**:
  - `batch_size` (int): 반환할 샘플 데이터의 크기.
- **반환값**:
  - `states`, `actions`, `rewards`, `next_states`, `dones`:
    - 각각 `torch.Tensor` 형식으로 반환되며, 학습 과정에서 입력 데이터로 사용.
- **특징**:
  - 무작위 샘플링을 통해 샘플 간 상관성을 줄여 안정성을 강화합니다.

---

### `__len__(self)`
- **설명**:
  - 버퍼에 저장된 경험 데이터의 개수를 반환합니다.
- **반환값**:
  - 현재 저장된 데이터의 개수 (`int`).

---